#### Setting up the Generative AI Notebook
* Create the notebook and install the library.
* we'll install the necessary library: huggingface_hub.

In [ ]:
# Cell 1: Install Hugging Face Library
%pip install huggingface_hub "requests>2.29"

#### Load libraries and API Token.

In [ ]:
# Cell 2: Import Libraries and Load API Token
import os
from huggingface_hub import InferenceClient

# Load the API token from the environment variable
try:
    hf_token = os.environ["HF_TOKEN"]
    print("Hugging Face token loaded successfully.")
except KeyError:
    print("ERROR: HF_TOKEN environment variable not set.")
    print("Please set the token and restart VS Code.")

# Initialize the Inference Client
# We will use a powerful, instruction-tuned model from Mistral AI.
MODEL_ID = "mistralai/Mistral-7B-Instruct-v0.2"
client = InferenceClient(model=MODEL_ID, token=hf_token)

* Why Mistral-7B-Instruct? 

This is a very popular and high-performing open-source model. It's excellent at following instructions (like our prompts) and generates high-quality, natural-sounding text, making it a perfect free alternative for our use case.

#### Step 5.3 (Revised): Adapting Prompts for the Model
Different models require different prompt structures to perform at their best. The Mistral-Instruct model uses a specific format with [INST] and [/INST] tags to know where the user's instruction begins and ends.

We will adapt our prompts from the original Phase 5 to this new format.



#### Create the Mistral-formatted prompt templates.

In [ ]:
# Cell 3: Define Mistral-formatted Prompt Templates

PROMPT_TEMPLATES = {
    "Proactive Support Call": """
    [INST]
    You are a friendly and empathetic Customer Success Manager at ConnectSphere.
    Your goal is to proactively offer help to a customer who might be struggling, based on their recent activity.
    DO NOT sell anything. Your tone should be supportive and genuinely helpful.

    Customer Data:
    - Name: {Name}
    - Subscription Tier: {SubscriptionTier}
    - Recent Support Tickets: {SupportTickets}

    Task:
    Write a short, personal email (around 80 words) to {Name}.
    1. Start by acknowledging them as a valued customer.
    2. Mention that you noticed they've been in touch with support recently.
    3. Offer a direct, no-pressure 15-minute call to see if you can help them achieve their goals with the product.
    4. End with a friendly closing.
    [/INST]
    """,

    "20% Discount Offer": """
    [INST]
    You are a helpful marketing specialist at ConnectSphere.
    Your goal is to offer a special discount to a customer we've identified as being at-risk, framing it as a 'thank you' for their loyalty.
    Your tone should be appreciative and value-focused, not desperate.

    Customer Data:
    - Name: {Name}
    - Subscription Tier: {SubscriptionTier}
    - Months with us (Tenure): {Tenure}

    Task:
    Write a compelling email (around 100 words) to {Name}.
    1. Thank them for being a customer for {Tenure} months.
    2. Mention that you want to offer them a special 20% discount on their {SubscriptionTier} plan for the next 3 months as a token of appreciation.
    3. Briefly hint at a recent product update or benefit relevant to their tier.
    4. Provide a clear (but fictional) call to action to claim the offer.
    [/INST]
    """,

    "Send Educational Content": """
    [INST]
    You are a product expert at ConnectSphere.
    Your goal is to re-engage a new customer who seems inactive by showing them how to get more value from the product.
    Your tone should be encouraging and informative.

    Customer Data:
    - Name: {Name}
    - Subscription Tier: {SubscriptionTier}
    - Usage Frequency (logins last 30 days): {UsageFrequency}

    Task:
    Write a helpful email (around 120 words) to {Name}.
    1. Welcome them again to the {SubscriptionTier} plan.
    2. Acknowledge that getting started can be overwhelming.
    3. Based on their low usage, suggest a specific, high-impact feature they might have missed. (e.g., 'Automated Reporting' for Standard/Premium, 'Project Templates' for Basic). Be creative.
    4. Provide a link to a (fictional) short video tutorial or guide on that feature.
    5. End with an encouraging message, inviting them to explore.
    [/INST]
    """
}

#### Step 5.4: Building the Personalization Function
Now we'll create our generation function using the huggingface_hub client.

Create the email generation function.

In [ ]:
# Cell 4: Create the Email Generation Function
def generate_personalized_email(customer_data, action):
    """
    Generates a personalized email using the Hugging Face Inference API.

    Args:
        customer_data (dict): A dictionary with the customer's info.
        action (str): The next-best-action to take.

    Returns:
        str: The generated email content, or an error message.
    """
    if action not in PROMPT_TEMPLATES:
        return "Error: No prompt template found for the given action."

    # Add a fake name for the example
    if 'Name' not in customer_data:
        customer_data['Name'] = fake.first_name()

    prompt = PROMPT_TEMPLATES[action].format(**customer_data)

    try:
        print(f"--- Calling Hugging Face Inference API for action: {action} ---")
        # Use the client to generate text
        # Note: The first time you run this for a model, it might take longer as the model loads on the server.
        response = client.text_generation(
            prompt,
            max_new_tokens=250,  # Max length of the generated email
            temperature=0.7,
            top_p=0.95,
            repetition_penalty=1.1,
        )
        return response.strip()

    except Exception as e:
        return f"An error occurred with the Hugging Face API: {e}"

# We need a fake name generator for this example
from faker import Faker
fake = Faker()

#### Step 5.5: Testing the Full Cycle
This step is identical in purpose to before, but now it will call our new Hugging Face-powered function.

Import the NBA engine and run the test.

In [ ]:
# Cell 5: Full End-to-End Test
# We need to import our recommendation logic from Phase 4
import sys
if './engine' not in sys.path:
    sys.path.append('./engine')
from nba_engine import recommend_action

# Create sample at-risk customers
customer_struggling = {
    'CustomerID': 'c_001', 'SupportTickets': 6, 'Tenure': 30,
    'UsageFrequency': 50, 'MonthlyRevenue': 100, 'SubscriptionTier': 'Premium'
}
customer_new_inactive = {
    'CustomerID': 'c_002', 'SupportTickets': 1, 'Tenure': 5,
    'UsageFrequency': 10, 'MonthlyRevenue': 25, 'SubscriptionTier': 'Basic'
}
customer_high_value = {
    'CustomerID': 'c_003', 'SupportTickets': 0, 'Tenure': 48,
    'UsageFrequency': 25, 'MonthlyRevenue': 120, 'SubscriptionTier': 'Premium'
}
customers = [customer_struggling, customer_new_inactive, customer_high_value]

# Process each customer
for customer in customers:
    # Phase 4: Get the recommended action
    action = recommend_action(customer)

    # Phase 5 (Revised): Generate the personalized email for that action
    email = generate_personalized_email(customer, action)

    print("\n" + "="*50)
    print(f"CUSTOMER PROFILE: {customer['CustomerID']} -> ACTION: {action}")
    print("="*50)
    print("GENERATED EMAIL (via Hugging Face):")
    print(email)
    print("="*50 + "\n")

#### Step 5.6: Saving the Personalization Logic
Let's update our personalization_engine.py script with this new, free-to-use logic.

Update the personalization engine script.

1. Open the engine/personalization_engine.py file in VS Code.
2. Replace its entire contents with the following code.

In [ ]:
# engine/personalization_engine.py

import os
from huggingface_hub import InferenceClient
from faker import Faker

# --- Configuration ---
# Load the token from environment variables
try:
    HF_TOKEN = os.environ["HF_TOKEN"]
except KeyError:
    raise Exception("HF_TOKEN environment variable not set. Cannot initialize engine.")

# Define the model to use
MODEL_ID = "mistralai/Mistral-7B-Instruct-v0.2"

# Initialize the clients/generators
CLIENT = InferenceClient(model=MODEL_ID, token=HF_TOKEN)
FAKE_GENERATOR = Faker()


# --- Prompt Templates (Mistral Format) ---
PROMPT_TEMPLATES = {
    "Proactive Support Call": """
    [INST]
    You are a friendly and empathetic Customer Success Manager at ConnectSphere. Your goal is to proactively offer help to a customer who might be struggling. Your tone should be supportive and genuinely helpful.
    Customer Data: Name: {Name}, Subscription Tier: {SubscriptionTier}, Recent Support Tickets: {SupportTickets}.
    Task: Write a short, personal email (around 80 words) to {Name}. Mention you noticed they've been in touch with support recently and offer a direct, no-pressure 15-minute call to help them achieve their goals.
    [/INST]
    """,
    "20% Discount Offer": """
    [INST]
    You are a helpful marketing specialist at ConnectSphere. Your goal is to offer a special discount to a valued customer, framing it as a 'thank you'. Your tone should be appreciative and value-focused.
    Customer Data: Name: {Name}, Subscription Tier: {SubscriptionTier}, Months with us (Tenure): {Tenure}.
    Task: Write a compelling email (around 100 words) to {Name}. Thank them for being a customer for {Tenure} months and offer a special 20% discount on their {SubscriptionTier} plan for the next 3 months. Provide a clear call to action.
    [/INST]
    """,
    "Send Educational Content": """
    [INST]
    You are a product expert at ConnectSphere. Your goal is to re-engage a new customer who seems inactive by showing them product value. Your tone should be encouraging and informative.
    Customer Data: Name: {Name}, Subscription Tier: {SubscriptionTier}, Usage Frequency (logins last 30 days): {UsageFrequency}.
    Task: Write a helpful email (around 120 words) to {Name}. Acknowledge that getting started can be overwhelming and, based on their low usage, suggest a specific, high-impact feature they might have missed. Provide a link to a fictional tutorial.
    [/INST]
    """
}


# --- Main Function ---
def generate_personalized_email(customer_data, action):
    """
    Generates a personalized email using the Hugging Face Inference API.

    Args:
        customer_data (dict): A dictionary with the customer's info.
        action (str): The next-best-action to take.

    Returns:
        str: The generated email content, or an error message.
    """
    if action not in PROMPT_TEMPLATES:
        return "Error: No prompt template found for the given action."

    if 'Name' not in customer_data:
        customer_data['Name'] = FAKE_GENERATOR.first_name()

    prompt = PROMPT_TEMPLATES[action].format(**customer_data)

    try:
        response = CLIENT.text_generation(
            prompt,
            max_new_tokens=250,
            temperature=0.7,
            top_p=0.95,
        )
        return response.strip()
    except Exception as e:
        # In a real app, you'd want to log this error
        print(f"Error calling Hugging Face API: {e}")
        return "We seem to be having trouble generating a message right now. Please try again later."